<a href="https://colab.research.google.com/github/andrewargatkiny/contests/blob/main/MIPT%20%26%20Rucode.net%20AI%20Festival%20entry%20challenge/ConvolutionalNN_taskMIPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
! cp -r /content/drive/MyDrive/image_classification /content

In [ ]:
! unzip /content/image_classification/data.zip

In [ ]:
!pwd

/content


In [ ]:
! ls /content/data/train/| wc -l

2366


In [ ]:
df_train = pd.read_csv('/content/image_classification/train.csv', dtype='string')
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.1, rotation_range=15,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator=datagen.flow_from_dataframe(
  dataframe=df_train,
  directory="/content/data/train/",
  x_col="file_nm",
  y_col="label",
  subset="training",
  batch_size=32,
  seed=45,
  shuffle=True,
  class_mode="categorical",
  target_size=(300,300))

Found 2130 validated image filenames belonging to 4 classes.


In [ ]:
df_train.label.value_counts()

2    903
1    538
0    478
3    447
Name: label, dtype: Int64

In [ ]:
validation_generator=datagen.flow_from_dataframe(
  dataframe=df_train,
  directory="/content/data/train/",
  x_col="file_nm",
  y_col="label",
  subset="validation",
  batch_size=32,
  seed=42,
  shuffle=True,
  class_mode="categorical",
  target_size=(300,300))

Found 236 validated image filenames belonging to 4 classes.


## Pretrained EfficientNet instantiation (not very good results)
Maybe due to dropout layer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
IMG_SIZE = 300
NUM_CLASSES = 4
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    #x = img_augmentation(inputs)
    model = EfficientNetB3(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

## Transfer learning with MobileNetV2 (95% accuracy on test set without tweaking)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
IMG_SIZE = 300
NUM_CLASSES = 4
def build_model(num_classes):
    inputs = layers.Input(shape=(None, None, 3))
    #x = img_augmentation(inputs)
    model = MobileNetV2(include_top=False, input_tensor=inputs, weights="imagenet", 
                           alpha=0.35, classes=NUM_CLASSES)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    #x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="MobileNetV2")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
with tf.device('/device:GPU:0'):
  model3 = build_model(num_classes=NUM_CLASSES)

  epochs = 10 
  hist = model3.fit(train_generator, epochs=epochs, 
                  validation_data=validation_generator, 
                  verbose=2)

Epoch 1/10
67/67 - 85s - loss: 0.5752 - accuracy: 0.7817 - val_loss: 0.4320 - val_accuracy: 0.8517
Epoch 2/10
67/67 - 80s - loss: 0.2405 - accuracy: 0.9127 - val_loss: 0.3004 - val_accuracy: 0.9153
Epoch 3/10
67/67 - 80s - loss: 0.1957 - accuracy: 0.9286 - val_loss: 0.2637 - val_accuracy: 0.9153
Epoch 4/10
67/67 - 80s - loss: 0.1761 - accuracy: 0.9423 - val_loss: 0.2135 - val_accuracy: 0.9322
Epoch 5/10
67/67 - 80s - loss: 0.1550 - accuracy: 0.9423 - val_loss: 0.1858 - val_accuracy: 0.9449
Epoch 6/10
67/67 - 80s - loss: 0.1380 - accuracy: 0.9549 - val_loss: 0.2142 - val_accuracy: 0.9364
Epoch 7/10
67/67 - 81s - loss: 0.1384 - accuracy: 0.9521 - val_loss: 0.2158 - val_accuracy: 0.9449
Epoch 8/10
67/67 - 82s - loss: 0.1145 - accuracy: 0.9601 - val_loss: 0.1927 - val_accuracy: 0.9492
Epoch 9/10
67/67 - 81s - loss: 0.1195 - accuracy: 0.9582 - val_loss: 0.1662 - val_accuracy: 0.9407
Epoch 10/10
67/67 - 81s - loss: 0.1000 - accuracy: 0.9643 - val_loss: 0.1857 - val_accuracy: 0.9407


In [ ]:
from tensorflow.keras.models import save_model, load_model
filepath = './saved_model2'
save_model(model3, filepath)

INFO:tensorflow:Assets written to: ./saved_model2/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
 !zip -r saved_model2.zip saved_model2/ 


  adding: saved_model2/ (stored 0%)
  adding: saved_model2/variables/ (stored 0%)
  adding: saved_model2/variables/variables.data-00000-of-00001 (deflated 11%)
  adding: saved_model2/variables/variables.index (deflated 77%)
  adding: saved_model2/saved_model.pb (deflated 92%)
  adding: saved_model2/keras_metadata.pb (deflated 95%)
  adding: saved_model2/assets/ (stored 0%)


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_ds= image_dataset_from_directory('/content/data/test/',
                                              labels=None,
                                              label_mode=None,
                                             shuffle=False,
                                            #rescale=1./255,
                                             batch_size=32,
                                             image_size=(300, 300))
file_paths = test_ds.file_paths

Found 2367 files belonging to 1 classes.


At first I made a mistake and didn't normalize the test images

In [ ]:
list(test_ds.as_numpy_iterator())

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = test_ds.map(lambda x: normalization_layer(x))
image_batch= next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0027692826 1.0


In [ ]:
datagen2=ImageDataGenerator(rescale=1./255.)
test_generator=datagen2.flow_from_directory(
  directory="/content/data/test/",
  class_mode=None,
  batch_size=32,
  shuffle=False,
  target_size=(300,300))

Found 0 images belonging to 0 classes.


In [ ]:
predictions = model3.predict(normalized_ds)

In [ ]:
preds_pd = pd.Series(np.argmax(predictions, axis=1), name='preds')
preds_pd.to_csv('predictions4.csv', index=False, header=False)

In [ ]:
predictions[:10]

array([[9.9753439e-01, 1.9057741e-03, 1.4211087e-05, 5.4569141e-04],
       [9.9999750e-01, 4.0101224e-08, 7.2242322e-08, 2.3997270e-06],
       [9.9953794e-01, 8.3297637e-06, 7.4252816e-06, 4.4633792e-04],
       [9.8144698e-01, 4.9148733e-04, 4.6363762e-03, 1.3425182e-02],
       [9.9720061e-01, 9.4031952e-07, 8.4367719e-05, 2.7140616e-03],
       [9.9976498e-01, 9.5686701e-05, 8.2493816e-06, 1.3109753e-04],
       [9.7165978e-01, 7.2329181e-07, 5.0299695e-05, 2.8289182e-02],
       [5.9333670e-01, 1.4630714e-01, 1.8627925e-01, 7.4076906e-02],
       [9.8673820e-01, 1.2157277e-06, 6.3197472e-06, 1.3254242e-02],
       [9.9905819e-01, 3.1157239e-07, 3.6208805e-06, 9.3781116e-04]],
      dtype=float32)

In [ ]:
preds_pd.head(60)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    2
12    0
13    0
14    3
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    1
23    0
24    1
25    0
26    1
27    3
28    0
29    1
30    1
31    1
32    0
33    3
34    3
35    1
36    3
37    1
38    0
39    3
40    0
41    3
42    3
43    0
44    3
45    0
46    3
47    0
48    0
49    1
50    0
51    3
52    0
53    0
54    0
55    3
56    0
57    0
58    0
59    0
Name: preds, dtype: int64

In [ ]:
train_generator.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3}

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

# Failed attempt to train CNN from scratch

In [ ]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Not connected to a TPU runtime. Using CPU/GPU strategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
from cloud_tpu_client import Client
c = Client()
c.configure_tpu_version(tf.__version__, restart_type="always")

In [ ]:
model2 = Sequential([
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(4)
])
with tf.device('/device:GPU:0'):
  model2.compile(optimizer='adam',
                loss="categorical_crossentropy",
                metrics=['accuracy'])
  epochs = 25 
  hist = model2.fit(train_generator, epochs=epochs, 
                  validation_data=validation_generator, 
                  verbose=2)